In [17]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os.path as op
from pprint import pprint as pp
import scipy.stats
import seaborn as sns
import sys
pth = op.abspath("../")
print pth
sys.path.insert(0, pth)
import teigen
reload(teigen)
print teigen.__path__

/Users/mjirik/projects/teigen
['/Users/mjirik/projects/teigen/teigen']


In [19]:
run_list = [
    #"compensation method",
    # "compensation method 1 sphere",
    # "compensation method 10 spheres",
    "compensation method 3 spheres various radius",
    # "compensation method 1 tube",
    # "compensation method 1 tube various radius",
    # "compensation method 1 tube various length",
    # "resolution tube"
    # "compensation method 5 tubes",
    # "best test", 
    #"fixed resolution"
]

In [20]:
tg = teigen.tgmain.Teigen()
config = tg.get_default_config()

In [21]:
# pp(config)
config

OrderedDict([('generators',
              OrderedDict([('Voronoi tubes',
                            OrderedDict([('element_number', 30),
                                         ('uniform_radius_distribution', True),
                                         ('normal_radius_distribution', False),
                                         ('fixed_radius_distribution', False),
                                         ('radius_distribution_minimum', 2.0),
                                         ('radius_distribution_maximum', 10.0),
                                         ('radius_distribution_mean', 5.0),
                                         ('radius_distribution_standard_deviation',
                                          5.0),
                                         ('random_generator_seed', 0)])),
                           ('Gensei', OrderedDict([('n_objects', 10)])),
                           ('Continuous tubes',
                            OrderedDict([('element_number', 3

In [22]:
np.arange(0, 10)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

# Prepare multiple configs


In [23]:
configs = []
tg = teigen.tgmain.Teigen()

In [24]:
run_label = "tube length"
if run_label in run_list:
    for length in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
        for rnd in [101, 102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = length
            gc["length_distribution_standard_deviation"] = 0.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = 20
            config["appearance"]["skip_volume_generation"] = True
            configs.append(config)

In [25]:
run_label = "tube length 2"
if run_label in run_list:
    for length in [5, 10, 15, 20, 25, 30, 35, 40, 45]:
        for rnd in [101, 102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = length
            gc["length_distribution_standard_deviation"] = 0.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = 20
            config["appearance"]["skip_volume_generation"] = True
            configs.append(config)

# Vepsaná, opsaná, 

In [26]:
run_label = "compensation method"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
        for rnd in [101]:
            for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 10
                gc["orientation_anisotropic"] = False
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 20
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

### 1 sphere

In [27]:
run_label = "compensation method 1 sphere"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 18, 20, 22, 24, 26, 28, 30, 34, 38, 42, 46, 50]:
        for rnd in [101, 102, 103]:
            for radius_method in [
                "inscribed", 
                #"circumscribed", 
                #"cylinder surface", 
                #"cylinder volume", 
                #"average", 
                "cylinder volume + sphere compensation"
            ]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 1
                gc["orientation_anisotropic"] = False
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 0.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

### 10 spheres

In [28]:
run_label = "compensation method 10 spheres"
if run_label in run_list:
    for resolution in [6, 10, 17, 25, 35, 50]:
        for rnd in [101]:
            for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 10
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 0.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

### 3 spheres various radius

In [29]:
run_label = "compensation method 3 spheres various radius"
import itertools
if run_label in run_list:
    resolutions = [25]
    for resolution, radius, rnd, radius_method in itertools.product(
        [25],
        [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50],
        [102, 103],
        ["inscribed", "cylinder surface", "cylinder volume", "cylinder volume + sphere compensation"]
    ):
    # for resolution in [25]:
    #     for radius in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    #         for rnd in [102, 103]:
    #             for radius_method in ["inscribed", "circumscribed", "cylinder surface", "cylinder volume", "average"]:
        config = tg.get_default_config()
        config["generator_id"] = 3
        config["output"]["note"] = run_label + " " + radius_method
        gc = config["generators"]["Unconnected tubes"]
        gc["element_number"] = 3
        gc["uniform_radius_distribution"] = False
        gc["fixed_radius_distribution"] = True
        gc["orientation_anisotropic"] = False
        gc["radius_distribution_mean"] = radius
        gc["radius_distribution_standard_deviation"] = 0.0
        gc["radius_distribution_maximum"] = 30.0
        gc["radius_distribution_minimum"] = 3.0
        gc["length_distribution_mean"] = 0.0
        gc["length_distribution_standard_deviation"] = 0.0
        gc["random_generator_seed"] = rnd
        config["postprocessing"]["measurement_resolution"] = resolution
        config["appearance"]["skip_volume_generation"] = True
        config["measurement"]["polygon_radius_selection_method"] = radius_method
        configs.append(config)

## 1 tube

In [30]:
run_label = "compensation method 1 tube"
if run_label in run_list:
    for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
        for rnd in [101]:
            for radius_method in ["cylinder surface", "cylinder volume", "cylinder volume + sphere compensation"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 1
                gc["uniform_radius_distribution"] = False
                gc["fixed_radius_distribution"] = True
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 0.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 20.0
                gc["length_distribution_standard_deviation"] = 0.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

## 1 tube various radius

In [31]:
run_label = "compensation method 1 tube various radius"
if run_label in run_list:
    # for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    for resolution in [25]:
        for radius in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
            for rnd in [101, 102]:
                for radius_method in ["cylinder surface", "cylinder volume", "cylinder volume + sphere compensation"]:
                    config = tg.get_default_config()
                    config["generator_id"] = 3
                    config["output"]["note"] = run_label + " " + radius_method
                    gc = config["generators"]["Unconnected tubes"]
                    gc["element_number"] = 1
                    gc["uniform_radius_distribution"] = False
                    gc["fixed_radius_distribution"] = True
                    gc["orientation_anisotropic"] = False
                    gc["radius_distribution_mean"] = radius
                    gc["radius_distribution_standard_deviation"] = 0.0
                    gc["radius_distribution_maximum"] = 60.0
                    gc["radius_distribution_minimum"] = 3.0
                    gc["length_distribution_mean"] = 20.0
                    gc["length_distribution_standard_deviation"] = 0.0
                    gc["random_generator_seed"] = rnd
                    config["postprocessing"]["measurement_resolution"] = resolution
                    config["appearance"]["skip_volume_generation"] = True
                    config["measurement"]["polygon_radius_selection_method"] = radius_method
                    configs.append(config)

## 1 tube various length

In [32]:
run_label = "compensation method 1 tube various length"
if run_label in run_list:
    # for resolution in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
    for resolution in [25]:
        for length in [6, 7, 8, 10, 12, 16, 20, 25, 30, 40, 50]:
            for rnd in [101, 102]:
                for radius_method in ["cylinder surface", "cylinder volume", "cylinder volume + sphere compensation"]:
                    config = tg.get_default_config()
                    config["generator_id"] = 3
                    config["output"]["note"] = run_label + " " + radius_method
                    gc = config["generators"]["Unconnected tubes"]
                    gc["element_number"] = 1
                    gc["uniform_radius_distribution"] = False
                    gc["fixed_radius_distribution"] = True
                    gc["orientation_anisotropic"] = False
                    gc["radius_distribution_mean"] = 20.0
                    gc["radius_distribution_standard_deviation"] = 0.0
                    gc["radius_distribution_maximum"] = 60.0
                    gc["radius_distribution_minimum"] = 3.0
                    gc["length_distribution_mean"] = length
                    gc["length_distribution_standard_deviation"] = 0.0
                    gc["random_generator_seed"] = rnd
                    config["postprocessing"]["measurement_resolution"] = resolution
                    config["appearance"]["skip_volume_generation"] = True
                    config["measurement"]["polygon_radius_selection_method"] = radius_method
                    configs.append(config)

# Všehochuť

In [33]:
run_label = "compensation method 5 tubes"
if run_label in run_list:
    for resolution in [8, 11, 13, 16, 30, 50]:
        for rnd in [107, 105]:
            for radius_method in [
                "cylinder surface", 
                "cylinder volume", 
                "cylinder volume + sphere compensation", 
                "cylinder surface + sphere compensation", 
                "best"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 5
                gc["uniform_radius_distribution"] = True
                gc["fixed_radius_distribution"] = False
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 2.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 30.0
                gc["length_distribution_standard_deviation"] = 2.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

In [34]:
# run_label = "resolution shape"
# if run_label in run_list:
#     for resolution in [5, 6, 7, 8, 10, 12, 14, 16, 20, 24, 28, 32, 40, 48]:
#         for rnd in [101, 102, 103, 104, 105]:
#             config = tg.get_default_config()
#             config["generator_id"] = 3
#             config["output"]["note"] = run_label
#             gc = config["generators"]["Unconnected tubes"]
#             gc["element_number"] = 10
#             gc["orientation_anisotropic"] = False
#             gc["length_distribution_mean"] = 0.0
#             gc["length_distribution_standard_deviation"] = 0.0
#             gc["random_generator_seed"] = rnd
#             config["postprocessing"]["measurement_resolution"] = resolution
#             config["appearance"]["skip_volume_generation"] = True
#             configs.append(config)

run_label = "resolution tube"
if run_label in run_list:
    for resolution in [6, 7, 9, 11, 13, 15, 19, 23, 28, 33, 39, 50]:
        for rnd in [102, 103]:
            config = tg.get_default_config()
            config["generator_id"] = 3
            config["output"]["note"] = run_label
            gc = config["generators"]["Unconnected tubes"]
            gc["element_number"] = 10
            gc["orientation_anisotropic"] = False
            gc["length_distribution_mean"] = 40.0
            gc["length_distribution_standard_deviation"] = 2.0
            gc["random_generator_seed"] = rnd
            config["postprocessing"]["measurement_resolution"] = resolution
            config["appearance"]["skip_volume_generation"] = True
            config["measurement"]["polygon_radius_selection_method"] = "cylinder volume + sphere compensation"
            configs.append(config)


## Best

In [35]:
run_label = "best test"
if run_label in run_list:
    for resolution in [8, 11, 13, 16, 30, 50]:
        for rnd in [106, 107, 108, 109, 110]:
            for radius_method in ["best"]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 5
                gc["uniform_radius_distribution"] = True
                gc["fixed_radius_distribution"] = False
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 2.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 30.0
                gc["length_distribution_standard_deviation"] = 2.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

In [36]:
run_label = "fixed resolution"
if run_label in run_list:
    for resolution in [25]:
        for rnd in [106, 107, 108, 109, 110, 111, 112, 115, 116, 117, 118, 119]:
            for radius_method in [
                # "inscribed", 
                # "average",
                "cylinder volume + sphere compensation", 
                "cylinder surface + sphere compensation", 
                #"best", 
            ]:
                config = tg.get_default_config()
                config["generator_id"] = 3
                config["output"]["note"] = run_label + " " + radius_method
                gc = config["generators"]["Unconnected tubes"]
                gc["element_number"] = 5
                gc["uniform_radius_distribution"] = True
                gc["fixed_radius_distribution"] = False
                gc["orientation_anisotropic"] = False
                gc["radius_distribution_mean"] = 20
                gc["radius_distribution_standard_deviation"] = 2.0
                gc["radius_distribution_maximum"] = 30.0
                gc["radius_distribution_minimum"] = 3.0
                gc["length_distribution_mean"] = 30.0
                gc["length_distribution_standard_deviation"] = 2.0
                gc["random_generator_seed"] = rnd
                config["postprocessing"]["measurement_resolution"] = resolution
                config["appearance"]["skip_volume_generation"] = True
                config["measurement"]["polygon_radius_selection_method"] = radius_method
                configs.append(config)

# Run multiple configs

In [37]:
#import psutil

#def print_number_of_opened_files():
#    for proc in psutil.process_iter():
#        print proc.open_files()

In [38]:

tg = teigen.tgmain.Teigen()
for i, cfg in enumerate(configs):
    # old failing
    # tg = teigen.tgmain.Teigen()
    # config = tg.get_default_config()
    tg.use_default_config()
    tg.update_config(**cfg)
    print (str(i) + " / " + str(len(configs)))  
    print(gc["random_generator_seed"])
    print(config["postprocessing"]["measurement_resolution"])
    tg.run()
    print("run finished")
    tg.step2()
    print("step2 finished")
    # del(tg)
    

0 / 88
103
25
0.0
0.0
0.0

/Users/mjirik/projects/teigen/teigen/geometry3d.py:800: RuntimeWarning: invalid value encountered in divide
  vector /= np.linalg.norm(vector)
/Users/mjirik/projects/teigen/teigen/tb_vtk.py:444: RuntimeWarning: invalid value encountered in divide
  br['direction'] = vv / nm.linalg.norm(vv)



0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
1 / 88
103
25
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
2 / 88
103
25
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
3 / 88
103
25
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
4 / 88
103
25
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
5 / 88
103
25
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
6 / 88
103
25
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 fi

/Users/mjirik/projects/teigen/teigen/geometry3d.py:15: RuntimeWarning: invalid value encountered in divide
  vector = length * vector / np.linalg.norm(vector)
/Users/mjirik/projects/teigen/teigen/geometry3d.py:342: RuntimeWarning: invalid value encountered in greater
  if np.all(node > (0 + radius)) and np.all(node < (areasize - radius)):
/Users/mjirik/projects/teigen/teigen/geometry3d.py:408: RuntimeWarning: invalid value encountered in divide
  _A = A / magA
/Users/mjirik/projects/teigen/teigen/geometry3d.py:409: RuntimeWarning: invalid value encountered in divide
  _B = B / magB
/Users/mjirik/projects/teigen/teigen/geometry3d.py:816: RuntimeWarning: invalid value encountered in less
  return np.all(position < 0)



0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
41 / 88
103
25
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
42 / 88
103
25
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
43 / 88
103
25
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
44 / 88
103
25
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333333333333
0.333333333333
step2 finished
45 / 88
103
25
0.0
0.0
0.0
0.0
0.0
0.333333333333
0.333333333333
0.333333333333
run finished
0.333333333333
0.333

ZeroDivisionError: float division by zero